In [1]:
from osgeo import gdal
import numpy as np
import pandas as pd
from stressor_utils import calculate_cell_area, resample_structured_grid, bin_layer, read_raster, classify_layer_area

In [2]:
rx2,ry2 = np.meshgrid(np.arange(0, 100, 1), np.arange(0, 100, 0.75))
rrx2, rry2, sqarea2 = calculate_cell_area(rx2, ry2, latlon=False)
sqarea2

array([[0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       ...,
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75]])

In [3]:
raster_filename = r"H:\Projects\C1308_SEAT\SEAT_outputs\plugin-output\Demo_20230822\calculated_stressor.tif"
receptor_filename = r"H:\Projects\C1308_SEAT\SEAT_inputs\plugin-input\DEMO structured\receptor\grainsize_receptor.tif"

# DF = bin_layer(raster_filename, receptor_filename=receptor_filename, receptor_names=None, latlon=True)
DF = bin_layer(raster_filename, 
            receptor_filename=receptor_filename, 
            receptor_names=None,  
            limit_receptor_range = [0, np.inf],
            latlon=True)

In [4]:
classified_raster = 'H:\Projects\C1308_SEAT\SEAT_outputs\plugin-output\Demo_20230822a\calculated_stressor_reclassified.tif'
DFc = classify_layer_area(classified_raster, 
        receptor_filename=receptor_filename, 
        at_values=[-2, -1, 0, 1, 2], 
        value_names=['Increased Deposition', 'Reduced Deposition','No Change', 'Reduced Erosion','Increased Erosion'], 
        limit_receptor_range=[0, np.inf],
        latlon=True)
DFc

,value,value name,"Area m2, receptor value 0.0","Count, receptor value 0.0","Area percent, receptor value 0.0","Area m2, receptor value 50.0","Count, receptor value 50.0","Area percent, receptor value 50.0","Area m2, receptor value 150.0","Count, receptor value 150.0","Area percent, receptor value 150.0","Area m2, receptor value 300.0","Count, receptor value 300.0","Area percent, receptor value 300.0","Area m2, receptor value 5000.0","Count, receptor value 5000.0","Area percent, receptor value 5000.0"
0,-2,Increased Deposition,1.412698e+04,1,0.076550,0.000000,0,0.0,0.000000e+00,0,0.0,2.402014e+05,7668,0.073307,1.060730e+05,2,0.581338
1,-1,Reduced Deposition,2.825707e+04,1,0.153116,0.000000,0,0.0,0.000000e+00,0,0.0,1.270734e+05,7668,0.038781,5.656944e+04,2,0.310032
2,0,No Change,1.834167e+07,1,99.387900,0.000000,0,0.0,0.000000e+00,0,0.0,4.613972e+06,7668,1.408136,1.625926e+05,2,0.891097
3,1,Reduced Erosion,6.352147e+04,1,0.344203,183513.950196,0,100.0,7.795038e+06,0,100.0,2.685567e+08,7668,81.960655,1.790699e+07,2,98.140118
4,2,Increased Erosion,7.055226e+03,1,0.038230,0.000000,0,0.0,0.000000e+00,0,0.0,5.412743e+07,7668,16.519121,1.412554e+04,2,0.077416


In [5]:
classified_raster = 'H:\Projects\C1308_SEAT\SEAT_outputs\plugin-output\Demo_20230822a\calculated_stressor_reclassified.tif'
DFd = classify_layer_area(classified_raster, 
        receptor_filename=None, 
        at_values=[-2, -1, 0, 1, 2], 
        value_names=['Increased Deposition', 'Reduced Deposition','No Change', 'Reduced Erosion','Increased Erosion'], 
        limit_receptor_range=None,
        latlon=True)
DFd

,value,value name,Area m2,Area percent
0,-2,Increased Deposition,3.604013e+05,0.096792
1,-1,Reduced Deposition,2.119000e+05,0.056910
2,0,No Change,2.311823e+07,6.208823
3,1,Reduced Erosion,2.945057e+08,79.094878
4,2,Increased Erosion,5.414861e+07,14.542598


In [6]:
limit_receptor_range = [0, np.inf]
rx, ry, z = read_raster(raster_filename)
rxm, rym, square_area = calculate_cell_area(rx, ry, latlon=True)

rrx, rry, receptor = read_raster(receptor_filename)
receptor = np.where((receptor>=np.min(limit_receptor_range)) & (receptor<=np.max(limit_receptor_range)), receptor, 0)
receptor = resample_structured_grid(rrx, rry, receptor, rxm, rym).flatten()

In [7]:
np.unique(receptor)

array([   0.,   50.,  150.,  300., 5000.], dtype=float32)

In [10]:
from shear_stress_module import run_shear_stress_stressor


out = run_shear_stress_stressor(
    dev_present_file=r'H:\Projects\C1308_SEAT\SEAT_inputs\plugin-input\DEMO structured\devices-present',
    dev_notpresent_file=r'H:\Projects\C1308_SEAT\SEAT_inputs\plugin-input\DEMO structured\devices-not-present',
    bc_file=r"H:\Projects\C1308_SEAT\SEAT_inputs\plugin-input\DEMO structured\boundary-condition\boundary-conditions.csv",
    crs=4326,
    output_path=r"H:\Projects\C1308_SEAT\SEAT_outputs\plugin-output\tests",
    receptor_filename= r'H:\Projects\C1308_SEAT\SEAT_inputs\plugin-input\DEMO structured\receptor\grainsize_receptor.tif'
)

h:\Projects\C1308_SEAT\Acoustics\seat-qgis-plugin\code\shear_stress_module.py:208: RuntimeWarning: All-NaN axis encountered
  tau_dev = np.nanmax(tau_dev, axis=1, keepdims=True) #max over time
h:\Projects\C1308_SEAT\Acoustics\seat-qgis-plugin\code\shear_stress_module.py:209: RuntimeWarning: All-NaN axis encountered
  tau_nodev = np.nanmax(tau_nodev, axis=1, keepdims=True) #max over time


AttributeError: 'NoneType' object has no attribute 'GetRasterBand'